In [81]:
from InstagramAPI import InstagramAPI
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
from pandas.io.json import json_normalize
import time
from datetime import datetime, timedelta, timezone
from instabot import Bot

In [17]:
# Login with InstaBOT
bot = Bot(max_likes_per_day = 0,
         max_unlikes_per_day  = 0,
         max_follows_per_day  = 0,
         max_unfollows_per_day  = 0,
         max_comments_per_day  = 0,
         max_likes_to_like  = 0,
         filter_users = False,
         filter_business_accounts  = False,
         filter_verified_accounts  = False,
         save_logfile= False)
bot.login(username="memes_you_see_before_bed", password="namanjain")

2020-07-20 22:10:10,643 - INFO - Instabot version: 0.117.0 Started
2020-07-20 22:10:10,647 - INFO - Not yet logged in starting: PRE-LOGIN FLOW!
2020-07-20 22:10:19,802 - INFO - Logged-in successfully as 'memes_you_see_before_bed'!
2020-07-20 22:10:19,803 - INFO - LOGIN FLOW! Just logged-in: True


True

In [3]:
# Login to the account with instaAPI
time.sleep(120)
def login_to_instagram(username, password):
    api = InstagramAPI(username, password)
    api.login()
    
    return api

api = login_to_instagram('memes_you_see_before_bed','namanjain')

Request return 405 error!
{'message': '', 'status': 'fail'}
Request return 404 error!
Login success!



In [6]:
# Get the data of all posts
def get_my_posts(api):
    '''Retrieve all posts from own profile'''
    my_posts = []
    has_more_posts = True
    max_id= ''

    while has_more_posts:
        api.getSelfUserFeed(maxid=max_id)
        if api.LastJson['more_available'] is not True:
            has_more_posts = False #stop condition

        max_id = api.LastJson.get('next_max_id','')
        my_posts.extend(api.LastJson['items']) #merge lists
        time.sleep(2) # slows down to avoid flooding

        if has_more_posts:
            print(str(len(my_posts)) + ' posts retrieved so far...')

    print('Total posts retrieved: ' + str(len(my_posts)))
    
    return my_posts

my_posts = get_my_posts(api)

18 posts retrieved so far...
36 posts retrieved so far...
54 posts retrieved so far...
72 posts retrieved so far...
90 posts retrieved so far...
108 posts retrieved so far...
126 posts retrieved so far...
144 posts retrieved so far...
162 posts retrieved so far...
180 posts retrieved so far...
198 posts retrieved so far...
216 posts retrieved so far...
234 posts retrieved so far...
252 posts retrieved so far...
270 posts retrieved so far...
288 posts retrieved so far...
306 posts retrieved so far...
324 posts retrieved so far...
342 posts retrieved so far...
360 posts retrieved so far...
378 posts retrieved so far...
Total posts retrieved: 387


In [7]:
# Get data of all likers
def get_posts_likers(api, my_posts):
    '''Retrieve all likers on all posts'''
    
    likers = []
    
    print('wait %.1f minutes' % (len(my_posts)*2/60.))
    for i in range(len(my_posts)):
        m_id = my_posts[i]['id']
        api.getMediaLikers(m_id)
        
        likers += [api.LastJson]
        
        # Include post_id in likers dict list
        likers[i]['post_id'] = m_id
        
        time.sleep(2)
    print('done')
    
    return likers


likers = get_posts_likers(api, my_posts) 

wait 12.9 minutes
done


In [8]:
# Get data of all commenters
def get_posts_commenters(api, my_posts):
    '''Retrieve all commenters on all posts '''
    
    commenters = []
    
    print('wait %.1f minutes' % (len(my_posts)*2/60.))
    for i in range(len(my_posts)):
        m_id = my_posts[i]['id']
        api.getMediaComments(m_id)
        
        commenters += [api.LastJson]
        
        # Include post_id in commenters dict list
        commenters[i]['post_id'] = m_id
            
        time.sleep(2)
    print('done')
    
    return commenters

commenters = get_posts_commenters(api, my_posts)

wait 12.9 minutes
done


In [10]:
# Convert all Json files into Pandas DataFrame
# Create dataframe of likers
def posts_likers_to_df(likers):
    '''Transforms likers list of dicts into pandas DataFrame'''
    
    # Normalize likers by getting the 'users' list and the post_id of each like
    df_likers = json_normalize(likers, 'users', ['post_id'])
    
    # Add 'content_type' column to know the rows are likes
    df_likers['content_type'] = 'like'
    
    return df_likers


# Create data frame of commenters
def posts_commenters_to_df(commenters):
    '''Transforms commenters list of dicts into pandas DataFrame'''
    
    # Include username and full_name of commenter in 'comments' list of dicts
    for i in range(len(commenters)):
        if len(commenters[i]['comments']) > 0: # checks if there is any comment on the post
            for j in range(len(commenters[i]['comments'])):
                # Puts username/full_name one level up
                commenters[i]['comments'][j]['username'] = commenters[i]['comments'][j]['user']['username']
                commenters[i]['comments'][j]['full_name'] = commenters[i]['comments'][j]['user']['full_name']
                
    # Create DataFrame
    # Normalize commenters to have 1 row per comment, and gets 'post_id' from parent 
    df_commenters = json_normalize(commenters, 'comments', 'post_id')
    
    # Get rid of 'user' column as we already handled it above
    
    return df_commenters


# Create Dataframe of posts
def posts_to_df(my_posts):
    return pd.DataFrame(my_posts)



df_posts = posts_to_df(my_posts)
df_likers = posts_likers_to_df(likers)
df_commenters = posts_commenters_to_df(commenters)

C:\Users\Aman\anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  import sys
C:\Users\Aman\anaconda3\lib\site-packages\ipykernel_launcher.py:29: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


# Data Processing

In [131]:
# Data of POSTS
df_posts.head()

In [132]:
# set the timing of post
df_posts['post_time'] = pd.to_datetime(df_posts['taken_at'], unit= 's') + timedelta(hours= 5, minutes= 30)

# Make media type more understandable
df_posts['media_type'] = df_posts['media_type'].map({
    1 : 'Image',
    2 : 'Video'
})

# If the media was applied with filter
df_posts['filter_type'] = df_posts['filter_type'].map({ 0 : 'No filter'})

# filter out caption
df_posts['caption'] = df_posts['caption'].apply(lambda x: x['text'] if x is not None else None)

# product type is the info if video on igtv or not
df_posts['on_igtv'] = df_posts['product_type']
df_posts['on_igtv'] = df_posts['on_igtv'].map({
    'igtv': True
})
df_posts['on_igtv'].fillna(False, inplace= True)

# fix null values
df_posts['filter_type'].fillna('Filter_used', inplace= True)
df_posts['has_audio'].fillna(False, inplace= True)
df_posts['video_duration'].fillna(0, inplace= True)
df_posts['view_count'].fillna(0, inplace= True)
df_posts['has_reshares'].fillna(False, inplace= True)
df_posts['title_on_igtv'] = df_posts['title'].fillna('Not_on_igtv')
df_posts['caption'].fillna('No caption', inplace= True)
df_posts['is_post_live'].fillna(False, inplace= True)

# make non audio video GIF
idx = df_posts[(df_posts['media_type'] == 'Video') & (df_posts['has_audio'] == False)]['media_type'].\
index
def make_gif():
    for i in idx:
        df_posts.loc[i, 'media_type'] = "GIF"
make_gif()


# allot proper datatypes
df_posts['can_viewer_reshare'] = df_posts['can_viewer_reshare'].astype('bool')
df_posts['comment_count'] = df_posts['comment_count'].astype('int16')
df_posts['like_count'] = df_posts['like_count'].astype('int16')
df_posts['view_count'] = df_posts['view_count'].astype('int16')


# Remove columns with no info to provide
to_drop = ['pk', 'device_timestamp', 'code', 'client_cache_key', 'image_versions2', 'original_height', 'original_width',\
          'boost_unavailable_reason', 'media_cropping_info', 'thumbnails', 'taken_at', 'has_more_comments',\
          'user', 'comment_threading_enabled', 'max_num_visible_preview_comments', 'preview_comments',\
          'can_view_more_preview_comments', 'inline_composer_display_condition', 'top_likers',\
          'facepile_top_likers', 'likers', 'photo_of_you', 'can_see_insights_as_brand', 'fb_user_tags',\
          'can_viewer_save', 'organic_tracking_token', 'is_dash_eligible', 'video_dash_manifest',\
          'video_codec', 'number_of_qualities', 'video_versions', 'product_type',\
          'nearly_complete_copyright_match', 'comment_likes_enabled', 'has_liked', 'title',\
           'can_viewer_reshare']
df_posts.drop(to_drop, axis= 1, inplace= True)

In [136]:
# Data of likers
df_likers.head()

,pk,username,full_name,is_private,profile_pic_url,profile_pic_id,is_verified,latest_reel_media,post_id,content_type
0,38580745270,peak_district_travels,Peak District Travels,False,https://instagram.fdel1-3.fna.fbcdn.net/v/t51....,2355385339886279082_38580745270,False,0,2357279277800629678_25061040410,like
1,39169629268,rubiminina,Rubi,False,https://instagram.fdel1-3.fna.fbcdn.net/v/t51....,2352030700525645836_39169629268,False,1595211342,2357279277800629678_25061040410,like
2,39390531310,pakka_mallu_artist,SreeKooz Art,False,https://instagram.fdel1-3.fna.fbcdn.net/v/t51....,2353816366154134381_39390531310,False,1595252547,2357279277800629678_25061040410,like
3,38730672650,queen.bgm01,queen.bgm,False,https://instagram.fdel1-3.fna.fbcdn.net/v/t51....,2347926243734529707_38730672650,False,1595221107,2357279277800629678_25061040410,like
4,3232061485,nupur.11_,Nupur,True,https://instagram.fdel1-3.fna.fbcdn.net/v/t51....,2356931630595436867_3232061485,False,0,2357279277800629678_25061040410,like


In [140]:
# pk is liker's ID
df_likers.rename(columns= { 'pk': 'liker_id'}, inplace= True)

# proper datatypes
df_likers['liker_id'] = df_likers['liker_id'].astype('object')

# Remave useless columns
to_drop = ['profile_pic_url', 'profile_pic_id', 'latest_reel_media', 'content_type']
df_likers.drop(to_drop, axis= 1, inplace= True)

In [146]:
# Data of commenters
df_commenters.head()

,pk,user_id,text,type,created_at,created_at_utc,content_type,status,bit_flags,did_report_as_spam,share_enabled,has_liked_comment,comment_like_count,inline_composer_display_condition,restricted_status,username,full_name,user.pk,user.username,user.full_name,user.is_private,user.profile_pic_url,user.profile_pic_id,user.is_verified,user.is_mentionable,user.latest_reel_media,user.latest_besties_reel_media,post_id
0,17860106378052034,38586540489,😂😂😂,0,1595247188,1595247188,comment,Active,0,False,False,False,0,always,0,itz_mimo.7,E S H A N samaddar,38586540489,itz_mimo.7,E S H A N samaddar,False,https://instagram.fdel1-3.fna.fbcdn.net/v/t51....,2355118948876147850_38586540489,False,True,0,0,2357271696930333070_25061040410
1,18113689921191086,38348756017,😂 Vry funnel 🤣,0,1595213342,1595213342,comment,Active,0,False,False,False,0,never,0,shwills_byshwetajais,Shweta Jais,38348756017,shwills_byshwetajais,Shweta Jais,False,https://instagram.fdel1-3.fna.fbcdn.net/v/t51....,2349857241913345777_38348756017,False,True,0,0,2356593389758882608_25061040410
2,17856974993087935,38348756017,Funny *,0,1595213377,1595213377,comment,Active,0,False,False,False,0,never,0,shwills_byshwetajais,Shweta Jais,38348756017,shwills_byshwetajais,Shweta Jais,False,https://instagram.fdel1-3.fna.fbcdn.net/v/t51....,2349857241913345777_38348756017,False,True,0,0,2356593389758882608_25061040410
3,17864421631911956,4629992719,Kaand with kunal,0,1594921039,1594921039,comment,Active,0,False,False,True,1,never,0,srivastava6146,Kunal Shrivastava,4629992719,srivastava6146,Kunal Shrivastava,True,https://instagram.fdel1-3.fna.fbcdn.net/v/t51....,2199598476639003635_4629992719,False,True,0,0,2354356869863278335_25061040410
4,17859707327033978,38098165617,He’s practical,0,1594712905,1594712905,comment,Active,0,False,False,False,0,never,0,anxiety_mewmew,,38098165617,anxiety_mewmew,,False,https://instagram.fdel1-3.fna.fbcdn.net/v/t51....,2344529762062823038_38098165617,False,True,0,0,2352941803339122855_25061040410


In [153]:
# comment timing in IST
df_commenters['comment_time'] = pd.to_datetime(df_commenters['created_at'], unit= 's') + timedelta(hours= 5, minutes= 30)

# proper datatypes
df_commenters['user_id'] = df_commenters['user_id'].astype('object')
df_commenters['comment_like_count'] = df_commenters['comment_like_count'].astype('int16')

# Remove useless columns
to_drop = ['pk', 'type', 'created_at', 'created_at_utc', 'content_type', 'status', 'bit_flags',\
          'share_enabled', 'inline_composer_display_condition', 'restricted_status', 'user.pk',\
          'user.username', 'user.full_name', 'user.profile_pic_url', 'user.profile_pic_id',\
          'user.is_mentionable', 'user.latest_reel_media', 'user.latest_besties_reel_media']
df_commenters.drop(to_drop, axis= 1, inplace= True)

In [158]:
# save data
df_posts.to_csv('posts.csv', index= False)
df_likers.to_csv('likers.csv', index= False)
df_commenters.to_csv('commenters.csv', index= False)

In [155]:
# logout
bot.logout()
api.logout()